# Setup

In [2]:
!pip install kaggle > /dev/null
!pip install torch torchvision > /dev/null
!conda install -c conda-forge tpot -y > /dev/null
!conda install -c conda-forge tpot xgboost dask dask-ml scikit-mdr skrebate -y > /dev/null

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

from tpot import TPOTClassifier

In [13]:
pd_clean_train = pd.read_csv("./data/clean/train.csv", index_col='PassengerId')
pd_clean_test = pd.read_csv("./data/clean/test.csv", index_col='PassengerId')
pd_sample_submission = pd.read_csv("../input/sample_submission.csv")

In [14]:
pd_clean_train.info()
pd_clean_train.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Survived     100000 non-null  int64  
 1   Pclass       100000 non-null  int64  
 2   Name         100000 non-null  object 
 3   Sex          100000 non-null  object 
 4   Age          96708 non-null   float64
 5   SibSp        100000 non-null  int64  
 6   Parch        100000 non-null  int64  
 7   Ticket       95377 non-null   object 
 8   Fare         99866 non-null   float64
 9   Cabin        32134 non-null   object 
 10  Embarked     99750 non-null   object 
 11  family       100000 non-null  int64  
 12  ticket_type  24655 non-null   object 
 13  ticket_num   94704 non-null   float64
 14  cabin_id     100000 non-null  object 
 15  surname      100000 non-null  object 
dtypes: float64(3), int64(5), object(8)
memory usage: 13.0+ MB


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,family,ticket_type,ticket_num,cabin_id,surname
PassengerId,,,,,,,,,,,,,,,,
0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S,2,NaN,209245.0,C,Oconnor


# Create Modeling Datasets

In [22]:
def format_dataset(pd_df):
    dataset_cols = [
        'Sex',
        'Embarked',
        'Pclass',
        'Age',
        'SibSp',
        'Parch',
        'Fare',
        'family',
        'ticket_type',
        'cabin_id',
        'Survived']
    
    return pd_df.loc[:, [col for col in dataset_cols if col in pd_df.columns]]


def one_hot_encode(pd_df):
    pd_df = pd.get_dummies(pd_df, columns=['Sex', 'Embarked', 'ticket_type', 'cabin_id'], dummy_na=True)
    pd_df = pd_df.drop('Sex_nan', axis=1)  # no null values exist in training or test set
    
    return pd_df

In [23]:
pd_model_train = (pd_clean_train
    .pipe(format_dataset)
    .pipe(one_hot_encode))
pd_model_test = (pd_clean_test
    .pipe(format_dataset)
    .pipe(one_hot_encode))

In [24]:
pd_model_train.head(1)

,Pclass,Age,SibSp,Parch,Fare,family,Survived,Sex_female,Sex_male,Embarked_C,...,cabin_id_A,cabin_id_B,cabin_id_C,cabin_id_D,cabin_id_E,cabin_id_F,cabin_id_G,cabin_id_T,cabin_id_n,cabin_id_nan
PassengerId,,,,,,,,,,,,,,,,,,,,,
0,1,NaN,2,0,27.14,2,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0


# AutoML

In [27]:
pipeline_optimizer = TPOTClassifier(
    max_time_mins=2, 
    max_eval_time_mins=0.04, 
    population_size=40,
    verbosity=2,
    random_state=3
)

In [ ]:
pipeline_optimizer.fit(pd_model_train.drop('Survived', axis=1), pd_model_train.Survived)

Imputing missing values in feature set
